In [1]:
import igl
#import meshplot as mp
import vedo as vd
import polyscope as ps
import numpy as np
import os 
import time 
from geometry.mesh import Mesh
from geometry.utils import *
from optimization.Planarity import Planarity
from optimization.HyperbolicLC import HyperbolicLC
from optimization.Optimizer import Optimizer
from optimization.LineCong import LineCong

vd.settings.default_backend = 'k3d'

## Initial Line Congruence test

### Visualization setup

In [ ]:
dir_path = os.getcwd()

data_path = dir_path+"/approximation/data/"

mv, mf = igl.read_triangle_mesh(os.path.join(data_path,"centers.obj" ))
tv, tf = igl.read_triangle_mesh(os.path.join(data_path,"test_remeshed.obj" ))
# Get vertex normals for test mesh
tn = igl.per_vertex_normals(tv, tf)

signs = np.sign(np.sum(tn * ([0,0,1]), axis=1))

# Fix normal directions
tn = tn * signs[:, None]

# Compute circumcenters and axis vectors for each triangle
p1, p2, p3 = tv[tf[:, 0]], tv[tf[:, 1]], tv[tf[:, 2]]

ct, rt, nt = circle_3pts(p1, p2, p3)

# Create dual mesh
mesh = Mesh()

mesh.make_mesh(tv,tf)

# Get inner vertices
inner_vertices = mesh.inner_vertices()

# Dual topology including outer faces
dual = mesh.dual_top()

# Dual topology without outer faces
dual_faces = [dual[vertex] for vertex in inner_vertices ]


# dual_faces = dualmesh.faces()
# # Draw vertex normals
# # vd.show(vd.Points(tv, c='red'), vd.Arrows(tv, tv + tn * 0.1, c='blue'))

# # Draw circles
# #circles = [ vd.Line(vd.Circle(ct, rt, c='k').orientation(nt), closed=True, lw=1.2, c='black') for ct, rt, nt in zip(ct, rt, nt) ]

# Create test mesh and center mesh 
mesh =  vd.Mesh((tv, tf), alpha = 0.8)
center = vd.Mesh((mv, mf), alpha = 0.9, c=[0.4, 0.4, 0.81])

# Create hexagonal mesh                            
h_pts = np.empty((len(tf), 3), dtype=np.float64)

# Intersect circumcircle axis with center mesh
for i in range(len(tf)):
    # Get points on circumcircle axis
    p0  = ct[i] - 10*nt[i]
    p1  = ct[i] + 10*nt[i]
    
    # Get intersection points
    h_pts[i,:] = np.array(center.intersect_with_line(p0, p1)[0])

# Create hexagonal mesh
hexmesh = vd.Mesh((h_pts, dual_faces), alpha = 0.5, c=[0.2, 0.4, 0.21])

# Draw hexagonal mesh
linesHex = []

for face in dual_faces:
        hex_face = []
        for vi in face:
            hex_face.append(h_pts[vi])
        linesHex.append(vd.Line(hex_face, closed=True, lw=1.8, c='white'))
            
# Draw edges center mesh
center_edges = center.clone().wireframe().lw(1.5).flat().c('black').alpha(1)

# Draw edges Triangular mesh
edges = mesh.clone().lw(1.5).wireframe().flat().c('k').alpha(0.5)

# Draw axis lines
# Lines vd.Lines(ct- 5*nt, ct + 5*nt, c='red', lw=0.1)
clines = vd.Lines(ct[:5]- 5*nt[:5], ct[:5] + 5*nt[:5], c='tomato', lw=0.1)

vd.show(vd.Points(tv, c='red'), mesh, center, vd.Points(h_pts, c='red'), linesHex, edges, clines)
 

### Optimization Test

In [21]:
# Define paths
dir_path = os.getcwd()
data_path = dir_path+"/approximation/data/" # data path
out_path = dir_path+"/outputs/" # output path

# Data of interest
k = 2

# Init data files
file_pts = open(os.path.join( out_path, "points_"+str(k)+".dat"), "w")
ref = open(os.path.join( out_path, "init_directions_"+str(k)+".dat"), "w")


# Load M mesh (centers of sphere mesh)
mv, mf = igl.read_triangle_mesh( os.path.join(data_path ,"centers.obj") ) 
center = vd.Mesh((mv, mf), alpha = 0.9, c=[0.4, 0.4, 0.81])


# Load test mesh
tv, tf = igl.read_triangle_mesh(os.path.join(data_path,  "test_remeshed_"+str(k)+".obj"))

# Create dual mesh
tmesh = Mesh()
tmesh.make_mesh(tv,tf)
# Get inner vertices
inner_vertices = tmesh.inner_vertices()
# Get outer vertices
outer_vertices = tmesh.boundary_vertices()

# Get vertex normals for test mesh
e_i = igl.per_vertex_normals(tv, tf)

# Fix normal directions
signs = np.sign(np.sum(e_i * ([0,0,1]), axis=1))
e_i = e_i * signs[:, None]

# Compute circumcenters and axis vectors for each triangle
p1, p2, p3 = tv[tf[:, 0]], tv[tf[:, 1]], tv[tf[:, 2]]

ct, rt, nt = circle_3pts(p1, p2, p3)

# Dual topology 
dual = tmesh.vertex_ring_faces_list()

# Dual topology without outer faces
dual_faces = [dual[vertex] for vertex in inner_vertices ]

# Create hexagonal mesh                            
h_pts = np.empty((len(tf), 3), dtype=np.float64)

# Intersect circumcircle axis with center mesh
for i in range(len(tf)):
    # Get points on circumcircle axis
    p0  = ct[i] - 10*nt[i]
    p1  = ct[i] + 10*nt[i]
    
    # Get intersection points
    h_pts[i,:] = np.array(center.intersect_with_line(p0, p1)[0])


# # Write data to file
for v  in tmesh.inner_vertices():
    file_pts.write(str(tmesh.vertices[v][0]) + " " + str(tmesh.vertices[v][1]) + " " + str(tmesh.vertices[v][2]) + "\n")
    ref.write(str(e_i[v][0]) + " " + str(e_i[v][1]) + " " + str(e_i[v][2]) + "\n")

file_pts.close()
ref.close()




# Optimization
# Init  X0 = [e_i | A| delta]
X = e_i.flatten() 

# Init constraints
linecong = LineCong()

linecong.initialize_constraint(X, len(tv), h_pts, dual, inner_vertices)
 
# Init optimizer
opt = Optimizer()
opt.initialize_optimizer(X)

for i in range(10):
    # Create file per iteration
    directions = open(os.path.join( out_path, "directions_"+str(k)+"_"+str(i)+".dat"), "w")

    # Compute J, r for each constraint
    linecong.compute(inner_vertices, dual, X)
    
    # Add constraints  
    opt.add_constraint(linecong)

    # Optimize
    opt.optimize("LM")

    e_i = opt.X.reshape(-1,3)   

    # Write data to file
    for v  in inner_vertices:   
        directions.write(str(e_i[v][0]) + " " + str(e_i[v][1]) + " " + str(e_i[v][2]) + "\n")
    directions.close()

# Print log
opt.print_log()
e_i = opt.X.reshape(-1,3)   
    
# Visualize line congruence
#congruence = vd.Lines(tv[inner_vertices[:5]]- 5*e_i[inner_vertices[:5]], tv[inner_vertices[:5]] + 5*e_i[inner_vertices[:5]], c=[0.8,0.5,0.5], lw=0.1)


Mesh Data Structure: |V| = 469, |F| = 858, |E| = 1326
      Energy
0  78.036198
1  10.887505
2   1.699542
3   0.549916
4   0.401698
5   0.382214
6   0.380081
7   0.379561
8   0.379438
9   0.379408


In [23]:
# Compute torsal directions 
torsal_dir  = np.zeros((len(tf), 6))
barycenters = np.zeros((len(tf), 3))

for i_t in range(len(tf)):
    # Get triangle
    t = tf[i_t]
    # Get vertices
    vi, vj, vk = tv[t] 
    # Get directions
    ei, ej, ek = e_i[t]

    # Compute edges
    eij = ej - ei
    eik = ek - ei

    # Compute vectors
    vij = vj - vi
    vik = vk - vi

    # Compute barycenter of directions
    ec = (ei + ej + ek)/3

    # Compute barycenter of vertices
    barycenters[i_t] = (vi + vj+ vk)/3

    # Compute coefficients
    g0 = eij@np.cross(vij, ec)
    g1 = eij@np.cross(vik, ec) + eik@np.cross(vij, ec)
    g2 = eik@np.cross(vik, ec)

    # Compute discriminant
    disc = g1**2 - 4*g0*g2

    # Compute torsal direction
    if disc < 0 and abs(disc) > 1e-6 :
        print("Discriminant is negative for face ", t, " with value ", disc, ".")
        t1 = np.zeros(3)
        t2 = np.zeros(3)

    elif disc < 0 and abs(disc) < 1e-6 :
        t1 = (-g1)*vij +  (2*g0)*vik
        t2 = (-g1)*vij +  (2*g0)*vik

        # Normalize
        t1 = t1/np.linalg.norm(t1)
        t2 = t2/np.linalg.norm(t2)

    else:
        t1 = (-g1 + np.sqrt(disc))*vij +  (2*g0)*vik
        t2 = (-g1 - np.sqrt(disc))*vij +  (2*g0)*vik

        # Normalize
        t1 = t1/np.linalg.norm(t1)
        t2 = t2/np.linalg.norm(t2)

    torsal_dir[i_t] = np.concatenate((t1, t2))

tpos = open(os.path.join( out_path, "torsal_pos_"+str(k)+".dat"), "w")
tdir = open(os.path.join( out_path, "torsal_dir_"+str(k)+".dat"), "w")

for i in range(len(tf)):
    tpos.write(str(barycenters[i][0]) + " " + str(barycenters[i][1]) + " " + str(barycenters[i][2]) + "\n")
    tdir.write(str(torsal_dir[i][0]) + " " + str(torsal_dir[i][1]) + " " + str(torsal_dir[i][2]) + " " + str(torsal_dir[i][3]) + " " + str(torsal_dir[i][4]) + " " + str(torsal_dir[i][5]) + "\n")

tpos.close()
tdir.close()

igl.write_triangle_mesh(os.path.join( out_path, "tri_"+str(k)+".obj"), tv, tf)

Discriminant is negative for face  [124 430 359]  with value  -1.2439212578412284e-06 .
Discriminant is negative for face  [174 226 350]  with value  -1.4378050400955208e-06 .
Discriminant is negative for face  [341 459  18]  with value  -1.3967412820632913e-05 .
Discriminant is negative for face  [176 425 285]  with value  -2.3583127146230013e-06 .


True

In [20]:
# Initialize polyscope
ps.init()


# Register a Mesh
v_tmesh = ps.register_surface_mesh("Mesh", tv, tf, smooth_shade=True)
ps.register_surface_mesh("Sphere Centers Mesh", h_pts, dual_faces, smooth_shade=True, edge_color=[0.8,0.5,0.5], edge_width=0.1)

# Draw circumcenter axis
# Register a point cloud
#v_cc = ps.register_point_cloud("Circumcenter ", tv, radius=0.01)

#print("ei :", e_i.shape)
#print("ct :", ct.shape)
# basic visualization
v_tmesh.add_vector_quantity("Axis", e_i, defined_on='vertices', enabled=True, radius=0.0001, length=5, color=(0.8, 0.0, 0.2))


# View the point cloud and mesh we just registered in the 3D UI
ps.show()

### Optimization 

### Load data

In [3]:
# Define paths
dir_path = os.getcwd()
data_path = dir_path+"/approximation/data/" # data path
out_path = dir_path+"/outputs/" # output path

# Data of interest
k = 2

# Init data files
file_pts = open(os.path.join( out_path, "points_"+str(k)+".dat"), "w")
ref = open(os.path.join( out_path, "init_directions_"+str(k)+".dat"), "w")

# Load M mesh (centers of sphere mesh)
mv, mf = igl.read_triangle_mesh( os.path.join(data_path ,"centers.obj") ) 

# Load test mesh
tv, tf = igl.read_triangle_mesh(os.path.join(data_path,  "test_remeshed_"+str(k)+".obj"))

# Create dual mesh
tmesh = Mesh()
tmesh.make_mesh(tv,tf)

# Get inner vertices
inner_vertices = tmesh.inner_vertices()

# Get vertex normals for test mesh
e_i = igl.per_vertex_normals(tv, tf)

# Fix normal directions
signs = np.sign(np.sum(e_i * ([0,0,1]), axis=1))
e_i = e_i * signs[:, None]

# Compute circumcenters and axis vectors for each triangle
p1, p2, p3 = tv[tf[:, 0]], tv[tf[:, 1]], tv[tf[:, 2]]

ct, rt, nt = circle_3pts(p1, p2, p3)

# Dual topology 
dual = tmesh.vertex_ring_faces_list()

# Dual topology without outer faces
dual_faces = [dual[vertex] for vertex in inner_vertices ]

# Create hexagonal mesh                            
h_pts = np.empty((len(tf), 3), dtype=np.float64)
center = vd.Mesh((mv, mf), alpha = 0.9, c=[0.4, 0.4, 0.81])

# Intersect circumcircle axis with center mesh
for i in range(len(tf)):
    # Get points on circumcircle axis
    p0  = ct[i] - 10*nt[i]
    p1  = ct[i] + 10*nt[i]
    
    # Get intersection points
    h_pts[i,:] = np.array(center.intersect_with_line(p0, p1)[0])

## Init Optimization

In [7]:
# # tv test
# tv = np.array([
#     [0,0,1],
#     [1,0,0],
#     [0,1,0],
#     ])

# # tf test
# tf = np.array([
#     [0,1,2]
#     ])

# # e_i test
# e_i = np.array([
#     [0,0,0.2],
#     [0,0.4,0.4],
#     [0.5,0.5,0.5],
#     ])

# e_i = e_i / np.linalg.norm(e_i, axis=1)[:, None]

# # Compute the directions at the barycenters


# # Compute the norms of ec
# nc = np.linalg.norm(ec, axis=1)

# # Compute the edge vectors per each face
vi, vj, vk = tv[tf[:,0]], tv[tf[:,1]], tv[tf[:,2]]


# # Compute the edge vectors per each face
vij = vj - vi
vik = vk - vi

# # Set up X 
# #X[:3*self.nV] = V.flatten()

# eij = e_i[tf[:,1]] - e_i[tf[:,0]]
# eik = e_i[tf[:,2]] - e_i[tf[:,0]]

# # A = [vij, eik, ec] + [eij, vik, ec], where [ , , ] denotes determinant
# # A = det1 +  det2
# eikXec = np.cross(eik, ec)
# vikXec = np.cross(vik, ec)

# det1 = np.sum(vij*eikXec, axis=1)
# det2 = np.sum(eij*vikXec, axis=1)

# # b = [eij, eik, ec]  c = [vij, vik, ec]

# b = np.sum(eij*eikXec, axis=1)
# c = np.sum(vij*vikXec, axis=1)

# A = det1 + det2 

# print(A**2 - 4*b*c - 1**2)

# variables = [e_i | A| delta]; e_i direction per vertex in T and A is one per each triangle in T, same for delta 
X = np.zeros(3*len(tv) + 2*len(tf) )

# Optimization
X[:3*len(tv)] = e_i.flatten()


# Init constraints ---------------------------------------------
# # Init LineCong
#linecong = LineCong()
# X variables, e_i dim, pts in hexagonal mesh, faces in hexagonal mesh, inner vertices indices.
#linecong.initialize_constraint(X, len(tv), h_pts, dual, inner_vertices)

# # Init Hyperbolic
hyp = HyperbolicLC()
X = hyp.initialize_constraint(X, tv, tf, e_i)


# Init optimizer
opt = Optimizer()
opt.initialize_optimizer(X)

# Compute J, r for each constraint
#linecong.compute(inner_vertices, dual, X)

# Add constraints  
#opt.add_constraint(linecong)
time_opt = []

for i in range(10):

    # Take time
    start = time.time()
    hyp.compute(X, tf)

    opt.add_constraint(hyp)

    # Optimize
    opt.optimize("LM")
    end = time.time()

    time_opt.append(end - start)


opt.print_log()

print(f"time:\n {time_opt}")

e_i = opt.X[:3*len(tv)].reshape(-1,3)

print(np.linalg.norm(e_i, axis=1))

eij = e_i[tf[:,1]] - e_i[tf[:,0]]
eik = e_i[tf[:,2]] - e_i[tf[:,0]]


ec = np.sum( e_i[tf], axis = 1) / 3
# A = [vij, eik, ec] + [eij, vik, ec], where [ , , ] denotes determinant
# A = det1 +  det2
eikXec = np.cross(eik, ec)
vikXec = np.cross(vik, ec)

det1 = np.sum(vij*eikXec, axis=1)
det2 = np.sum(eij*vikXec, axis=1)

# b = [eij, eik, ec]  c = [vij, vik, ec]

b = np.sum(eij*eikXec, axis=1)
c = np.sum(vij*vikXec, axis=1)

A = det1 + det2 

print("final:", A**2 - 4*b*c)

print(f" real A : {A} \t opt A : {opt.X[3*len(tv):3*len(tv)+len(tf)]}")

print(f" step 5: {opt.X[:5]}")
# print(f" step 5 delta: {opt.X[-5:]}")
 

          Energy
0  536214.648664
1  534617.032594
2  533024.137525
3  531435.957096
4  529852.483199
5  528273.706359
6  527014.152461
7  525757.592033
8  524504.019025
9  523253.427222
time:
 [16.453320503234863, 14.464795589447021, 13.67778730392456, 14.403748989105225, 14.97875428199768, 18.28896951675415, 17.19410252571106, 21.405291318893433, 14.16629409790039, 15.57926058769226]
[1.00361101 1.00556462 0.99851517 1.00217869 1.00338967 0.99999193
 0.99789531 0.99997864 0.99994279 1.00005088 1.00009091 0.99983894
 0.99896696 0.99965317 0.99870042 1.00139416 1.00057006 1.00001506
 1.00006232 1.00038609 1.00082593 1.00136397 1.00009857 0.98206015
 1.00007317 0.99987863 1.00000895 1.00003272 1.00048481 1.00662129
 1.0009742  0.9897499  0.99906253 0.99744648 1.00017942 0.99924458
 1.00248295 1.00462127 1.00051056 1.00684262 1.00006334 1.00200841
 1.00270984 0.9893572  1.00118538 0.9992232  1.00129584 1.00120648
 1.00007791 1.00001318 0.9999967  0.99784483 1.00099905 1.00068228
 1.00027

In [5]:
print(opt.X[-5:])
opt.print_log()

[0.9962556  0.99625561 0.99625561 0.99625561 0.99625561]
         Energy
0    858.354786
1    860.535232
2    869.912134
3    940.831957
4  84358.007391


## Minimize Library test

In [ ]:
from scipy import optimize


def energy(x0, ei_dim, cicj, dual_faces):

    # energy \sum_{f \in F} \sum_{cj,ci \in E(f)} || e_f (cj - ci)  ||^2
    cf = dual_faces
    
    # Init energy value
    total_energy = 0

    # Get directions
    ei = x0.reshape(ei_dim, 3)

    # Loop over faces
    for f in range(len(cf)):
        # Define Jacobian
        diff = np.dot(cicj[f], ei[f])

        # Define residual
        total_energy += np.sum( diff**2, axis=0)
    
    diff2 = np.sum(ei * ei, axis=1) - 1

    total_energy += np.sum(diff2**2, axis=0)

    return total_energy

# Define paths
dir_path = os.getcwd()
data_path = dir_path+"/approximation/data/" # data path
out_path = dir_path+"/outputs/" # output path

# Data of interest
k = 3

# Load M mesh (centers of sphere mesh)
mv, mf = igl.read_triangle_mesh( os.path.join(data_path ,"centers.obj") ) 
center = vd.Mesh((mv, mf), alpha = 0.9, c=[0.4, 0.4, 0.81])


# Load test mesh
tv, tf = igl.read_triangle_mesh(os.path.join(data_path,  "test_remeshed_"+str(k)+".obj"))

# Create test mesh
test_mesh =  vd.Mesh((tv, tf), alpha = 0.8)

# Draw edges Triangular mesh
edges = test_mesh.clone().lw(1.5).wireframe().flat().c('k').alpha(0.5)

# Create dual mesh
tmesh = Mesh()
tmesh.make_mesh(tv,tf)
# Get inner vertices
inner_vertices = tmesh.inner_vertices()
# Get outer vertices
outer_vertices = tmesh.boundary_vertices()

# Get vertex normals for test mesh
e_i = igl.per_vertex_normals(tv, tf)

# Fix normal directions
signs = np.sign(np.sum(e_i * ([0,0,1]), axis=1))
e_i = e_i * signs[:, None]

# Compute circumcenters and axis vectors for each triangle
p1, p2, p3 = tv[tf[:, 0]], tv[tf[:, 1]], tv[tf[:, 2]]

ct, rt, nt = circle_3pts(p1, p2, p3)

# Dual topology 
dual = tmesh.dual_top()

# Dual topology without outer faces
dual_faces = [dual[vertex] for vertex in inner_vertices ]

# Create hexagonal mesh                            
h_pts = np.empty((len(tf), 3), dtype=np.float64)

# Intersect circumcircle axis with center mesh
for i in range(len(tf)):
    # Get points on circumcircle axis
    p0  = ct[i] - 10*nt[i]
    p1  = ct[i] + 10*nt[i]
    
    # Get intersection points
    h_pts[i,:] = np.array(center.intersect_with_line(p0, p1)[0])

# Compute cij
cij = []
# Loop over faces
for f in range(len(dual_faces)):
    # Get face
    face = dual_faces[f]

    # Get vertices
    v0 = h_pts[face]
    v1 = np.roll(h_pts[face], -1, axis=0)
 
    cicj = (v1 - v0)/np.linalg.norm(v1 - v0, axis=1)[:, None]
    # Define direction
    cij.append(cicj)

dim = len(inner_vertices)


opt = optimize.minimize(energy, e_i[inner_vertices].flatten(), args=(dim, cij, dual_faces), options={'disp': True})


In [ ]:
e_i = opt.x.reshape(-1,3)

minfile = open(os.path.join( out_path, "min_directions_"+str(k)+".dat"), "w")

for v  in range(len(tmesh.inner_vertices())):   
    minfile.write(str(e_i[v][0]) + " " + str(e_i[v][1]) + " " + str(e_i[v][2]) + "\n")
minfile.close()



### Test Circumcircle

In [ ]:

# Get a list of random points
#p = np.random.rand(10,3)

p1 = np.random.rand(10,3)
p2 = np.random.rand(10,3)
p3 = np.random.rand(10,3)

c, rad, n = circle_3pts(p1,p2,p3)
print(f"centers: {c}\t rad: {rad}\t n: {n}")
circles = []
for ci in range(len(c)):
    print(f"centers: {c[ci]}\t rad: {rad[ci]}\t n: {n[ci]}")
    if ci%2 == 0:
        circles.append( vd.Circle(c[ci], r=rad[ci], c='red', alpha = 0.5).orientation(n[ci])) 
    else:
        circles.append( vd.Circle(c[ci], r=rad[ci], c="green", alpha = 0.5).orientation(n[ci])) 
pts1 = vd.Points(np.array( np.vstack((p1, p2, p3))), r = 10, c = 'black')
#pts2 = vd.Points(np.array([p1[1], p2[1], p3[1]]), r = 10, c = 'green')

vd.show(pts1,*circles)


## Sample Planarity Opt

In [ ]:
# Create four quads in 3D that merge in a vertex
v = np.array(
    [
        [ 0.01,  0.01,  0.8],
        [ 1.01,  0.03,  0.01],
        [-1.02,  0.02,  0.01],
        [ 0.01,  1.1, -0.2],
        [ 0.01, -1.3, -0.3],
        [-1.02,  1.01,  0.1],
        [ 1.01,  1.02,  0.2],
        [-1.04, -1.03, -0.3],
        [ 1.05, -1.04,  0.1],
    ]
   )
fcs = np.array([[0, 1, 6, 3], [2, 0, 3, 5], [7, 4, 0, 2], [4, 8, 1, 0]])

# Make mesh
m = Mesh()
m.make_mesh(v, fcs)

mesh1 = vd.Mesh((v, fcs), alpha = 0.8, c='r')

X = np.zeros((3*m.V+3*m.F), dtype=np.float64)

X[:3*m.V] = m.vertices.flatten()

# Init constraints
planarity = Planarity()
planarity.initialize_constraint(m, X)


# init optimizer
opt = Optimizer()

opt.initialize_optimizer(X)

for i in range(10):

    # Compute J, r for each constraint
    planarity.compute(m, X)
    
    # Add constraints  
    opt.add_constraint(planarity)
    opt.optimize("LM")
 

mesh2 = vd.Mesh((opt.X[:3*m.V].reshape(m.V,3),  fcs), alpha = 0.9, c='b')

print(opt.energy)

vd.show(mesh1, mesh2)



In [25]:

v = np.array(
    [
        [ 0.01,  0.01,  0.8],
        [ 1.01,  0.03,  0.01],
        [-1.02,  0.02,  0.01],
        [ 0.01,  1.1, -0.2],
        [ 0.01, -1.3, -0.3],
        [-1.02,  1.01,  0.1],
        [ 1.01,  1.02,  0.2],
        [-1.04, -1.03, -0.3],
        [ 1.05, -1.04,  0.1],
    ]
   )
fcs = np.array([[0, 1, 6, 3], [2, 0, 3, 5], [7, 4, 0, 2], [4, 8, 1, 0]])

nv = np.array([[-0.08452795, -0.5184518 ,  0.1204968 ],
       [ 0.09325778, -0.5417252 , -0.14323919],
       [-0.39126414,  0.38199818,  0.21744937],
       [ 0.05301311,  0.4957555 , -0.02630325],
       [-0.04160952, -0.68063384,  0.11461049],
       [-0.64098644,  0.96492183,  0.3065665 ],
       [ 0.29607683,  0.33823755, -0.46029344],
       [-0.24634174, -0.29367885,  0.3622692 ],
       [ 0.5331171 , -1.4693466 , -0.43155265]]
       )


mesh1 = vd.Mesh((v, fcs), alpha = 0.8, c='r')
mesh2 = vd.Mesh((nv, fcs), alpha = 0.8, c='b')


vd.show(mesh1, mesh2)

Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], backgro…

In [ ]:
v, f = igl.read_triangle_mesh("models/catenoid_def_1.obj")
mesh = om.read_trimesh("models/catenoid_def_1.obj")
mp.plot(v, f, c=v[:, 0])

In [ ]:
def iso_stereo(n, h):
    # Isotropic model stereographic projection
    # n: normal vector
    # h: distance center to plane
    # return point in I3

    # Concatenate n[1:2] and h
    return np.vstack((n[1:2],v))/(1+ n[2])

In [ ]:
a = np.array([[1, 2, 3],[5,4,2] , [9,7,2]])
b = np.array([19,22,89])
# Add b as a column to a
c = np.column_stack((a,b))
print(a[:, 0:2])

In [ ]:
# Get face normals
nf = igl.per_face_normals(v, f, np.array([0.0, 0.0, 0.0]))
# Get indices v1 of each face
iv1 = f[:, 0]
# Get vertices of idv1
v1 = v[iv1, :]
# Compute dot product of each face normal with v1
h = np.sum(nf * v1, axis=1)

# Get Isotropic point per face
nv = np.column_stack((nf[:, 0:2], h)) / (1 + nf[:, 2, None])

nf = np.empty((0, 3), dtype=np.int32)

for fi in range(len(f)):
    aux = np.empty((0, ), dtype=np.int32)
    for adf in mesh.ff(mesh.face_handle(fi)):
        aux = np.hstack((aux, [adf.idx()]))
    if len(aux) == 3:
        nf = np.vstack((nf, aux))

mp.plot(nv, nf[:2])

In [ ]:
from scipy.sparse.linalg import spsolve
from scipy.sparse import csr_matrix

v, f = igl.read_triangle_mesh("models/Hall.obj")

# Draw the mesh
n = igl.per_vertex_normals(v, f) * 0.5 + 0.5
c = np.linalg.norm(n, axis=1)
p = mp.plot(v, f, c, return_plot=True) # plot

# Compute Cuvature properties
v1, v2, k1, k2 = igl.principal_curvature(v, f)



# Compute focal surfaces
rho1 = 100
rho2 = 1/k2

step = 0.001

lamb = 0.1 * np.ones(v.shape[0], dtype=np.float64)

J = np.eye(v.shape[0], dtype=np.float64) 

# Precalculate intermediate states
vs = [v]
cs = [c]
for i in range(30):
    # Compute the gradient of the focal surface
    H = csr_matrix(J.T@J)
    b = - J.T@(lamb - rho1)
    delta = spsolve(H, b)
    lamb = lamb + step * delta
    # Update the mesh
    v = v + lamb[:,None]*n

    # Recompute the normals
    n = igl.per_vertex_normals(v, f) * 0.5 + 0.5
    c = np.linalg.norm(n, axis=1)
    # Add to the list
    # p.update_object(vertices=v, colors=c)
    vs.append(v)
    cs.append(c)
    

# Add interactive visulization
@mp.interact(level=(0, 9))
def mcf(level=0):
    p.update_object(vertices=vs[level], colors=cs[level])
    vs.append(v)
    cs.append(c)
p